<a href="https://colab.research.google.com/github/ttogle918/ds-section1-sprint2/blob/master/n122-chi-square-test/n122a-chi-square-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 2*

# 📝 Assignment

# 카이제곱검정


## 1. 공공데이터 조작 연습. 

국가 통계 포털에서 아래 이미지를 참조하여

<https://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01#SelectStatsBoxDiv>

**2020년 8월**에 해당하는 규모별 미분양현황 에 대한 데이터셋을 생성하세요. 

<img src='https://i.imgur.com/aZc4UJO.png' width = 500>

- `60m이하`, `60~85m`, `85m초과`의 3개 규모와 (column)
- `서울`, `대전`, `대구`, `부산` 4개의 지역을 포함해야합니다. (row)
- `민간부문`만 포함합니다
- 데이터가 없는 경우는 0으로 처리하세요.

이후 데이터셋을 colab으로 불러오세요. 이때 변수의 이름은 `df`를 사용합니다.


In [1]:
# google 드라이브에 로컬 파일 업로드
from google.colab import files, drive
# uploaded = files.upload()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
col = ['~60m2', '60~85m2', '85m2~']
row = ['서울', '대전', '대구', '부산']

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import io
# df = pd.read_csv(io.BytesIO(uploaded["규모별_미분양현황_20210917135130.csv"]), encoding = "CP949", engine='python')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/code_states/규모별_미분양현황_20210917135130.csv", encoding = "CP949", engine='python')
df

,시도(1),부문(1),2020. 08,2020. 08.1,2020. 08.2
0,시도(1),부문(1),60㎡이하,60∼85㎡,85㎡초과
1,서울,민간부문,54,2,0
2,부산,민간부문,577,735,142
3,대구,민간부문,143,1437,44
4,대전,민간부문,782,1,0


In [4]:
df.fillna(0)

,시도(1),부문(1),2020. 08,2020. 08.1,2020. 08.2
0,시도(1),부문(1),60㎡이하,60∼85㎡,85㎡초과
1,서울,민간부문,54,2,0
2,부산,민간부문,577,735,142
3,대구,민간부문,143,1437,44
4,대전,민간부문,782,1,0


In [5]:
df = df.rename(
    columns = {
        '시도(1)' : '시도',
        '부문(1)' : '부문'
    }
)
df = df.drop('부문', axis = 1)

df.columns = ['지역'] + col
df = df[1:]
df = df.set_index(['지역'])
df.columns.name = '구분'
df = df.astype(int)
df

구분,~60m2,60~85m2,85m2~
지역,,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


## 2. 지역에 대해서 one-sample chi-square test를 실행, 해당 결과를 `chi1`에 저장 후 설명해보세요.

예시) 만약 **9월달 데이터**를 기준으로 한다면
```python
 [52+2+0, 590+665+142, 113+1061+42, 772+1+0]
```
을 비교 하게 될 것입니다.




In [6]:
from scipy.stats import chisquare
def convert(string) :
  value = value.apply(int)
  return value.sum()

df['sum'] = df.sum(axis=1)
chi1 = chisquare(df['sum'], axis=None)
chi1

Power_divergenceResult(statistic=1564.4572376818994, pvalue=0.0)

p-value가 0.0으로 나올 때 : 절삭된 것(0에 가까운 수, python에서 지정해 놓은 수를 넘거아면(너무 작으면) 자른다)


p-value가 0.05보다 크면 귀무가설을 기각할 수 없다. -> 1:1

p-value가 0.05보다 작으면 귀무가설을 기각한다.
-> 각 지역의 미분양현황은 기대와 동일하지 않다.(같지 않다.)

신뢰도가 95%라는 전제
p-value < 0.05, 귀무가설 기각 -> 대립가설을 채택

p-value > 0.05, 귀무가설 기각할 수 없음

## 3. 지역과 규모에 대해서 two-sample chi-square test를 실행, 해당 결과를 `chi2`에 저장 후 설명해보세요.

예시) **9월달 데이터**를 기준으로 한다면

| |-60 | 60-85 | 85- |
|:-:|:-:|:-:|:-:|
|서울|52|2|0|
|대전|772|1|0|
|대구|113|1061|42|
|부산|590|665|142|

에 대해서 검정해야 할 겁니다.




+ 해당 데이터는 이미 crosstab이 되어있다.

-> 아닐 때는 해당 코드 이용
```
obs = pd.crosstab(small_food['income'], small_food['pay_meal_out'])
```


In [7]:
from scipy.stats import chi2_contingency

chi2 = chi2_contingency(df)
chi2

(2064.576731417199,
 0.0,
 9,
 array([[  22.24559612,   31.09522594,    2.65917794,   56.        ],
        [ 577.59101353,  807.36533061,   69.04365586, 1454.        ],
        [ 645.12228746,  901.76155221,   77.11616033, 1624.        ],
        [ 311.04110288,  434.77789124,   37.18100587,  783.        ]]))

### 4. 2번에 대해서 NumPy 를 사용하여 (Scipy를 사용하지 않고) $\chi^2$ test 시행 후 2번의 결과와 비교해보세요. 

- `obs`, `exp`, `chi`라는 변수를 사용해야합니다.

In [8]:
df = df.drop('sum', axis=1)
df

구분,~60m2,60~85m2,85m2~
지역,,,
서울,54,2,0
부산,577,735,142
대구,143,1437,44
대전,782,1,0


In [47]:
sum_obs = np.sum(df)
sum_obs
sum_obs.dtypes

dtype('int64')

In [58]:
df_area_sum = []
df_area_sum = df.sum(axis=1).to_frame()
df_area_sum.astype(int)

df_size_sum = df.sum(axis = 0).to_frame()
df_size_sum.astype(int)
df_size_sum

,0
구분,
~60m2,1556
60~85m2,2175
85m2~,186


In [59]:
exp = []
for i in df_area_sum.index :
  t_list = []
  for j in df_size_sum.index :
    # print(df_area_sum.loc[i] * df_size_sum.loc[j])
    t_list.append( int(df_area_sum.loc[i]) * int(df_size_sum.loc[j]) / sum_obs)
    
  exp.append(t_list)
exp

[[0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64], [0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64], [0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64], [0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64, 0         NaN
  60~85m2   NaN
  85m2~     NaN
  ~60m2     NaN
  dtype: float64]]

## 🔥 도전 과제

아래 세가지를 다하면 됩니다.

### 1. Function

4번에서 사용한 one sample chisquare test를 함수의 형태로 변경하세요.

In [ ]:
v1 = [18,22,20,15,23,22]
v2 = [5,23,26,19,24,23]

def myChisq(value):
    return None# 내용을 채우세요.

myChisq(v1) 

# chisquare값과 pvalue를 출력해야합니다.

myChisq(v2)

### 2. ANOVA

아래 링크를 참조하여 ANOVA 에 대한 글을 읽고

<https://partrita.github.io/posts/ANOVA-python/>

다음 `4개 그룹에 대해서 평균의 차이가 있는지`에 대한 가설 검정을 시행하세요.

A : `38 33 35 92 76 97 88 41 11  9`

B : `18 52 62 48 30 40 87 12 97 82`

C :  `28 90  5 49 66 73 96 80  4 17`

D : ` 8 99  4 12  7 64 18 10  9 20`



In [ ]:
# 이 곳에 답안을 작성하시길 바랍니다